In [1]:
from rich import print
from pydantic import BaseModel, Field

from aimq.worker import Worker
from aimq.helpers import assign, const, pick, orig, echo, select
from aimq.tools.supabase import WriteRecord, ReadRecord, ReadFile, WriteFile, Enqueue
from aimq.tools.pdf import PageSplitter

from aimq.clients.supabase import supabase
from aimq.clients.mistral import mistral



In [2]:
worker = Worker()

"""Attachment details."""
class AttachmentDetails(BaseModel):
    summary: str = Field(..., description="Summary of the attachment")
    sentiment: str = Field(..., description="Sentiment of the attachment as a range from -1.00 to 1.00")

In [3]:
@worker.task()
def example(data: dict):
    path = data.get("storagePath")
    attachment_id = data.get("attachmentId")
    supabase.client.table("attachments").update({"status": "processing"}).eq("id", attachment_id).execute()

    file_data = supabase.client.storage.from_("attachments").download(path)
    uploaded_file = mistral.client.files.upload(file={"file_name": path, "content": file_data}, purpose="ocr")
    signed_url = mistral.client.files.get_signed_url(file_id=uploaded_file.id)
    ocr_result = mistral.client.ocr.process(
        model="mistral-ocr-latest",
        document={"type": "document_url", "document_url": signed_url.url}
    )
    # serialize ocr_result
    ocr_result = ocr_result.model_dump()
    # combine all pages into one string
    ocr_content = "\n".join([page["markdown"] for page in ocr_result["pages"]])

    print(ocr_content)
    
    details = mistral.client.chat.parse(
        messages=[
            {"role": "system", "content": "You are a summarizer. Given some markdown, summarize it and analyze the sentiment. sentiment is on a scale of -1.00 to 1.00 where -1.00 is very negative and 1.00 is very positive. 0 is neutral but should be avoided."},
            {"role": "user", "content": ocr_content}
        ],
        model="mistral-large-latest",
        response_format=AttachmentDetails,
        temperature=0.0
    )
    supabase.client.table("attachments").update({"content": ocr_result, "status": "processed"}).eq("id", attachment_id).execute()
    return {
        "signed_url": signed_url.url,
        "file_id": uploaded_file.id,
        "attachment_id": attachment_id,
        "details": details.model_dump() 
    }

worker.send("example", {
  "fileSize": 38366,
  "metadata": {
    "eTag": "\"ab6bb464d6d465973193aec6160da55b\"",
    "size": 38366,
    "mimetype": "application/pdf",
    "cacheControl": "no-cache",
    "lastModified": "2025-09-16T14:09:39.369Z",
    "contentLength": 38366,
    "httpStatusCode": 200
  },
  "mimeType": "application/pdf",
  "queuedAt": "2025-09-16T14:09:39.400713+00:00",
  "channelId": "8ed1a67c-763e-4ada-a492-2fc16694d989",
  "messageId": "4ccdcf31-c397-44e3-9036-08ea404c133a",
  "storagePath": "de7d6ea0-5a28-49ad-a44c-29df0153d973/original.pdf",
  "workspaceId": "dcec4c63-047e-45e9-bd13-6870c21a45b3",
  "attachmentId": "de7d6ea0-5a28-49ad-a44c-29df0153d973"
})


61

In [4]:
worker.start(block=False)

In [ ]:
worker.log(block=False)
worker.stop()